In [ ]:
!git clone https://github.com/sathishkumar67/Face-Recognition-using-Resnet.git
!mv /kaggle/working/Face-Recognition-using-Resnet/* /kaggle/working/
!pip install --upgrade pip
!pip install -r requirements.txt
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

  Using cached tensorflow-2.19.0-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
  Using cached absl_py-2.2.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-5.29.4-cp310-abi3-win_amd64.whl.metadata (592 bytes)
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached wrapt-1.17.2-cp312-cp312-win_amd64.whl.metadata (6.5 kB)
  Using cached grpcio-1.71.0-cp312-cp312-win_amd64.whl.metadata (4.0 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.9.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached h

In [ ]:
import os
import random
import shutil
import cv2
from mtcnn import MTCNN
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import Dataset
from huggingface_hub import hf_hub_download
from siamese_resnet import unzip_file
import torchvision.transforms as transforms
from tqdm import tqdm

In [2]:
DATASET_REPO_ID = "pt-sk/Face_Recognition_Dataset"
DATASET_FILENAME_IN_REPO = "Face Recognition Dataset.zip"
DATASET_REPO_TYPE = "dataset"
LOCAL_DIR = os.getcwd()

In [ ]:
# Download the dataset from Hugging Face Hub
hf_hub_download(repo_id=DATASET_REPO_ID, filename=DATASET_FILENAME_IN_REPO, repo_type=DATASET_REPO_TYPE, local_dir=LOCAL_DIR)

# Unzip the dataset
unzip_file(os.path.join(LOCAL_DIR, DATASET_FILENAME_IN_REPO), LOCAL_DIR)

In [ ]:
# triplet loss function
# This function computes the triplet loss for a batch of anchor, positive, and negative samples.
class TripletLoss(nn.Module):
    def __init__(self, margin=0.5):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative):
        # Compute pairwise distances
        distance_positive = F.pairwise_distance(anchor, positive)
        distance_negative = F.pairwise_distance(anchor, negative)
        
        # Calculate triplet loss
        losses = F.relu(distance_positive - distance_negative + self.margin)
        return losses.mean()
    


In [ ]:
class SiameseResNet(nn.Module):
    def __init__(self, embedding_dim=256):
        super(SiameseResNet, self).__init__()
        # Load pretrained ResNet18
        self.backbone = torchvision.models.resnet18(weights="IMAGENET1K_V1", progress=True)
        
        # Replace the final fully connected layer
        self.backbone.fc = nn.Linear(512, embedding_dim)  # 512 -> 256

    def forward(self, x):
        return self.backbone(x)
    
    def print_parameters_count(self):
        total_params = sum(p.numel() for p in self.parameters()) / 1e6  # Convert to millions
        # Print the number of parameters in millions
        print(f"Total parameters: {total_params:.2f}m")
    
model = SiameseResNet(embedding_dim=256)
# model.print_parameters_count()

# pass a sample image through the model
dummy_input = torch.randn(1, 3, 224, 224)  # batch size of 1, 3 channels, 224x224 image
output = model(dummy_input)
# output.shape  # should be (1, 256) since we changed the final layer to output 256 features

In [ ]:
import os
import random
from PIL import Image
from torch.utils.data import Dataset
from collections import defaultdict

class TripletFaceDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.id_to_images = defaultdict(list)
        
        # Populate identities and their images
        for identity in os.listdir(root_dir):
            identity_dir = os.path.join(root_dir, identity)
            if os.path.isdir(identity_dir):
                images = [os.path.join(identity_dir, img) 
                        for img in os.listdir(identity_dir) 
                        if img.endswith(('.jpg', '.png'))]
                if len(images) >= 2:  # Ensure at least 2 images per identity
                    self.id_to_images[identity] = images
        self.identities = list(self.id_to_images.keys())

    def __len__(self):
        return len(self.identities) * 10  # Adjust based on your needs

    def __getitem__(self, idx):
        # Anchor and positive from the same identity
        anchor_id = self.identities[idx % len(self.identities)]
        anchor_img_path, positive_img_path = random.sample(self.id_to_images[anchor_id], 2)
        
        # Negative from a different identity
        negative_id = random.choice(self.identities)
        while negative_id == anchor_id:
            negative_id = random.choice(self.identities)
        negative_img_path = random.choice(self.id_to_images[negative_id])
        
        # Load and transform images
        anchor = Image.open(anchor_img_path).convert('RGB')
        positive = Image.open(positive_img_path).convert('RGB')
        negative = Image.open(negative_img_path).convert('RGB')
        
        if self.transform:
            anchor = self.transform(anchor)
            positive = self.transform(positive)
            negative = self.transform(negative)
        
        return anchor, positive, negative

In [6]:
train_dataset = TripletFaceDataset(root_dir='sample_dataset')

In [7]:
train_dataset.__len__()

3500

In [ ]:

# ====================== Usage Example ======================
if __name__ == "__main__":
    random.seed(SEED)
    np.random.seed(SEED)
    
    # Initialize generator
    generator = TripletDatasetGenerator(DATA_ROOT)
    
    # Create splits
    splits = generator.create_splits()
    
    # Generate triplets for each split
    train_triplets = generator.generate_triplets(splits['train'])
    val_triplets = generator.generate_triplets(splits['val'])
    test_triplets = generator.generate_triplets(splits['test'])
    
    # Create datasets
    train_dataset = TripletDataset(train_triplets)
    val_dataset = TripletDataset(val_triplets)
    test_dataset = TripletDataset(test_triplets)
    
    # Create dataloaders (4 workers for optimal IO)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, 
                            num_workers=4, pin_memory=True)

In [ ]:
# Example usage:
for batch in train_loader:
    anchors = batch['anchor']  # numpy array (H, W, 3)
    positives = batch['positive']
    negatives = batch['negative']
    
    # Convert to tensors and normalize in your training loop:
    # anchors_tensor = torch.from_numpy(anchors).permute(0, 3, 1, 2).float() / 255.0

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms

# Define transforms
transform = transforms.Compose([
    transforms.Resize(100),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Initialize dataset and dataloader
dataset = TripletFaceDataset(root_dir="path/to/dataset", transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Initialize model, loss, and optimizer
model = SiameseResNet(embedding_dim=256)
criterion = TripletLoss(margin=0.5)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(10):
    for batch in dataloader:
        anchor, positive, negative = batch
        anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)
        
        # Forward pass
        anchor_emb = model(anchor)
        positive_emb = model(positive)
        negative_emb = model(negative)
        
        # Compute loss
        loss = criterion(anchor_emb, positive_emb, negative_emb)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")